In [589]:
import numpy
import numpy as np
import pandas as pd
import re
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [590]:
# extract data from log file by the special keyword
def read_log(file, model_name):
    line_list = []
    with open(file) as f:
        while True:
            lines = f.readline()
            line_list.append(lines)
            if not lines:
                break
    # print(line_list)
    if model_name == 'Mlp':
        keyline = "INFO - Average testing results among all repeated 80-20 holdouts:\n"
    for i in range(len(line_list)):
        if line_list[i] == keyline:
            key_index = i
    srcc = line_list[key_index + 1]
    krcc = line_list[key_index + 2]
    plcc = line_list[key_index + 3]
    rmse = line_list[key_index + 4]
    return srcc, krcc, plcc, rmse

def get_value(coef, model_name):
    if model_name == 'Mlp':
        pattern = r"INFO - (\w+): (\d+\.\d+) \(std: (\d+\.\d+)\)"

    matches = re.findall(pattern, coef)
    coef_value = "N/A" # Default value
    for match in matches:
        metric, value, std_dev = match
        value = format(float(value), '.4f')
        std_dev = format(float(std_dev), '.4f')
        plusmius = u"\u00B1"
        coef_value = f"{value} ({plusmius}{std_dev})"
    return coef_value

def process_logs(data_name, feats_name, model_name, log_path, select_criteria):
    data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = [], [], [], [], [], []
    for data in data_name:
        for feat in feats_name:
            if data == 'lsvq_test':
                log_file = f'{log_path}/lsvq_train_{feat}_{model_name}_{select_criteria}_kfold.log'
            elif data == 'lsvq_test_1080p':
                log_file = f'{log_path}/lsvq_train_{feat}_{model_name}_{select_criteria}_1080p.log'
            else:
                log_file = f'{log_path}/{data}_{feat}_{model_name}_{select_criteria}.log'
            if not os.path.exists(log_file):
                continue  # skip if the file doesn't exist
            with open(log_file, "r", encoding="UTF-8") as file:
                srcc, krcc, plcc, rmse = read_log(log_file, model_name)
                srcc = get_value(srcc, model_name)
                krcc = get_value(krcc, model_name)
                plcc = get_value(plcc, model_name)
                rmse = get_value(rmse, model_name)
    
            data_list.append(data)
            feats_list.append(feat)
    
            srcc_list.append(srcc)
            krcc_list.append(krcc)
            plcc_list.append(plcc)
            rmse_list.append(rmse)
    return data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list

def read_csv_log(data_name, feats_name, log_path, select_criteria, is_finetune):
    data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = [], [], [], [], [], []
    for feat in feats_name:
        for data in data_name:
            if is_finetune == True:
                log_file = f'{log_path}/{data}_{feat}_{select_criteria}_finetune.csv'
                ft_name = f'{feat}_finetune'
            else:
                log_file = f'{log_path}/{data}_{feat}_{select_criteria}_wo_finetune.csv'
                ft_name = f'{feat}_wo_finetune'
            # print(log_file)
            ft_df = pd.read_csv(log_file)
            selected_data = ft_df[ft_df['SELECT_CRITERIC'] == select_criteria]
            data_list.append(data)
            feats_list.append(ft_name)
            srcc_list.append(selected_data['SRCC'].iloc[0])
            krcc_list.append(selected_data['KRCC'].iloc[0])
            plcc_list.append(selected_data['PLCC'].iloc[0])
            rmse_list.append(selected_data['RMSE'].iloc[0])
    return data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list

def create_results_dataframe(data_list,  feats_list, srcc_list, krcc_list, plcc_list, rmse_list):
    df_results = pd.DataFrame(columns=['DATASET', 'FEATS_NAME', 'SRCC (±STD)', 'KRCC (±STD)', 'PLCC (±STD)', 'RMSE (±STD)'])

    df_results['DATASET'] = data_list
    df_results['FEATS_NAME'] =  feats_list
    df_results['SRCC (±STD)'] = srcc_list
    df_results['KRCC (±STD)'] = krcc_list
    df_results['PLCC (±STD)'] = plcc_list
    df_results['RMSE (±STD)'] = rmse_list
    return df_results

In [591]:
model_name = 'Mlp'
select_criteria = 'byrmse'
print(f'Model_name: {model_name}, Select_criteria: {select_criteria}')

Model_name: Mlp, Select_criteria: byrmse


ablation study on clip embeddings ----- sampled frame per half second, without any hints for quality and artifact embeddings

semantic_embs: image embeddings + quality embeddings + artifact embeddings

In [592]:
log_path = f'../log/ablation_study/semantic_embs_wo_hint/'
feats_name = ['img_embs', 'quality_embs', 'artifact_embs', 'semantic_embs_wo_hint']
data_name = ['konvid_1k']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = process_logs(data_name, feats_name, model_name, log_path, select_criteria)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,konvid_1k,img_embs,0.7785 (±0.0266),0.5873 (±0.0257),0.8002 (±0.0203),0.3879 (±0.0152)
1,konvid_1k,quality_embs,N/A,N/A,0.1166 (±0.0491),0.6352 (±0.0158)
2,konvid_1k,artifact_embs,0.1965 (±0.0755),0.1305 (±0.0524),0.2110 (±0.0655),0.6231 (±0.0181)
3,konvid_1k,semantic_embs_wo_hint,0.7673 (±0.0270),0.5758 (±0.0257),0.7904 (±0.0224),0.3901 (±0.0170)


ablation study on clip embeddings ----- sampled frame per half second, with metadata hints for quality and artifact embeddings

semantic_embs: image embeddings + quality embeddings + artifact embeddings

In [593]:
log_path = f'../log/ablation_study/semantic_embs/'
feats_name = ['img_embs', 'quality_embs', 'artifact_embs', 'content_embs', 
              'img_add_quality_embs', 'img_add_artifact_embs', 'img_add_content_embs', 'semantic_embs', 'semantic_embs_w_content']
data_name = ['konvid_1k']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = process_logs(data_name, feats_name, model_name, log_path, select_criteria)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,konvid_1k,img_embs,0.7784 (±0.0263),0.5898 (±0.0256),0.8043 (±0.0209),0.3866 (±0.0155)
1,konvid_1k,quality_embs,0.6312 (±0.0373),0.4641 (±0.0278),0.7920 (±0.0204),0.3888 (±0.0160)
2,konvid_1k,artifact_embs,0.7353 (±0.0441),0.5409 (±0.0382),0.7634 (±0.0349),0.4113 (±0.0313)
3,konvid_1k,content_embs,0.4089 (±0.0450),0.2786 (±0.0315),0.4511 (±0.0439),0.5742 (±0.0182)
4,konvid_1k,img_add_quality_embs,0.8295 (±0.0223),0.6434 (±0.0211),0.8707 (±0.0140),0.3171 (±0.0137)
5,konvid_1k,img_add_artifact_embs,0.8709 (±0.0159),0.6889 (±0.0178),0.8838 (±0.0111),0.3007 (±0.0127)
6,konvid_1k,img_add_content_embs,0.7527 (±0.0262),0.5607 (±0.0235),0.7830 (±0.0202),0.4018 (±0.0150)
7,konvid_1k,semantic_embs,0.9025 (±0.0148),0.7240 (±0.0193),0.9219 (±0.0071),0.2493 (±0.0090)
8,konvid_1k,semantic_embs_w_content,0.8924 (±0.0171),0.7098 (±0.0222),0.9185 (±0.0083),0.2518 (±0.0105)


ablation study on multimodal features

In [594]:
log_path = f'../log/ablation_study/features/'
feats_name = ['semantic_embs', 'slowfast', 'swint', 'semantic_embs_add_slowfast', 'semantic_embs_add_swint']
data_name = ['cvd_2014', 'konvid_1k', 'live_vqc', 'youtube_ugc_h264','lsvq_test', 'lsvq_test_1080p', 'finevd']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = process_logs(data_name, feats_name, model_name, log_path, select_criteria)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,cvd_2014,semantic_embs,0.9156 (±0.0345),0.7613 (±0.0352),0.9244 (±0.4067),8.8238 (±3.5570)
1,cvd_2014,slowfast,0.8250 (±0.0492),0.6386 (±0.0582),0.8348 (±0.0498),11.3901 (±1.5673)
2,cvd_2014,swint,0.8578 (±0.0272),0.6818 (±0.0361),0.8872 (±0.0298),9.9918 (±1.2139)
3,cvd_2014,semantic_embs_add_slowfast,0.9228 (±0.0207),0.7724 (±0.0314),0.9405 (±0.0230),7.4847 (±1.1450)
4,cvd_2014,semantic_embs_add_swint,0.9319 (±0.0184),0.7761 (±0.0336),0.9410 (±0.0189),7.1729 (±1.1270)
5,konvid_1k,semantic_embs,0.9021 (±0.0159),0.7280 (±0.0208),0.9217 (±0.0075),0.2490 (±0.0099)
6,konvid_1k,slowfast,0.7701 (±0.0228),0.5817 (±0.0210),0.7818 (±0.0233),0.4025 (±0.0157)
7,konvid_1k,swint,0.8611 (±0.0204),0.6756 (±0.0236),0.8600 (±0.0186),0.3244 (±0.0161)
8,konvid_1k,semantic_embs_add_slowfast,0.9092 (±0.0135),0.7394 (±0.0184),0.9214 (±0.0094),0.2532 (±0.0113)
9,konvid_1k,semantic_embs_add_swint,0.9286 (±0.0110),0.7676 (±0.0169),0.9373 (±0.0077),0.2261 (±0.0113)


FineVD feature ablation on different MOS Dimensions

ablation study on clip embeddings ----- semantic_embs: image embeddings + quality embeddings + artifact embeddings

In [595]:
log_path = f'../log/ablation_study/semantic_embs_finevd_ablation/'
feats_name = ['img_embs', 'quality_embs', 'artifact_embs', 'content_embs', 
              'img_add_quality_embs', 'img_add_artifact_embs', 'img_add_content_embs', 'semantic_embs', 'semantic_embs_w_content']
data_name = ['finevd']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = process_logs(data_name, feats_name, model_name, log_path, select_criteria)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,finevd,img_embs,0.8036 (±0.0146),0.6111 (±0.0148),0.8172 (±0.0118),7.4840 (±0.1654)
1,finevd,quality_embs,0.8162 (±0.0068),0.6159 (±0.0103),0.8693 (±0.0080),6.4194 (±3.1701)
2,finevd,artifact_embs,0.8121 (±0.0090),0.5943 (±0.0104),0.8398 (±0.0173),7.0197 (±0.3762)
3,finevd,content_embs,0.4011 (±0.0195),0.2753 (±0.0140),0.4090 (±0.0205),11.7691 (±0.1804)
4,finevd,img_add_quality_embs,0.8988 (±0.0057),0.7131 (±0.0089),0.9107 (±0.0056),5.3655 (±0.1378)
5,finevd,img_add_artifact_embs,0.8940 (±0.0067),0.7085 (±0.0094),0.9011 (±0.0065),5.6229 (±0.1550)
6,finevd,img_add_content_embs,0.7943 (±0.0156),0.5999 (±0.0155),0.8069 (±0.0131),7.6421 (±0.1744)
7,finevd,semantic_embs,0.9008 (±0.0064),0.7208 (±0.0098),0.9194 (±0.0056),5.0729 (±0.1249)
8,finevd,semantic_embs_w_content,0.8962 (±0.0058),0.7118 (±0.0083),0.9169 (±0.0045),5.1767 (±0.1091)


In [596]:
log_path = f'../log/ablation_study/semantic_embs_finevd_ablation/dimension/'
feats_name = ['mos_color_img_embs', 'mos_color_quality_embs', 'mos_color_artifact_embs', 'mos_color_camp-vqa', 
              'mos_noise_img_embs', 'mos_noise_quality_embs', 'mos_noise_artifact_embs', 'mos_noise_camp-vqa', 
              'mos_artifact_img_embs', 'mos_artifact_quality_embs', 'mos_artifact_artifact_embs', 'mos_artifact_camp-vqa', 
              'mos_blur_img_embs', 'mos_blur_quality_embs', 'mos_blur_artifact_embs', 'mos_blur_camp-vqa', 
              'mos_temporal_img_embs', 'mos_temporal_quality_embs', 'mos_temporal_artifact_embs', 'mos_temporal_camp-vqa']
data_name = ['finevd']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = process_logs(data_name, feats_name, model_name, log_path, select_criteria)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,finevd,mos_color_img_embs,0.8041 (±0.0136),0.6118 (±0.0137),0.8284 (±0.0112),6.5752 (±0.1902)
1,finevd,mos_color_quality_embs,0.7678 (±0.0116),0.5786 (±0.0126),0.8042 (±0.0186),6.9687 (±0.3390)
2,finevd,mos_color_artifact_embs,0.7496 (±0.0131),0.5406 (±0.0120),0.7650 (±0.0204),7.6079 (±0.3320)
3,finevd,mos_color_camp-vqa,0.8930 (±0.0560),0.7116 (±0.0676),0.8836 (±0.0508),5.6059 (±0.9205)
4,finevd,mos_noise_img_embs,0.7626 (±0.0170),0.5690 (±0.0157),0.7265 (±0.0225),7.7429 (±0.3624)
5,finevd,mos_noise_quality_embs,0.7448 (±0.0104),0.5559 (±0.0106),0.7670 (±0.0205),7.2521 (±0.3839)
6,finevd,mos_noise_artifact_embs,0.7637 (±0.0118),0.5560 (±0.0108),0.7356 (±0.0185),7.6715 (±0.3342)
7,finevd,mos_noise_camp-vqa,0.8733 (±0.0309),0.6797 (±0.0385),0.8187 (±0.0342),6.5147 (±0.5566)
8,finevd,mos_artifact_img_embs,0.8062 (±0.0132),0.6122 (±0.0136),0.8142 (±0.0108),7.2169 (±0.1149)
9,finevd,mos_artifact_quality_embs,0.7610 (±0.0104),0.5588 (±0.0122),0.8273 (±0.3560),7.0232 (±1.5322)


intra-dataset results of CAMP-VQA

In [597]:
log_path = f'../log/best_model/'
feats_name = ['camp-vqa']
data_name = ['cvd_2014', 'konvid_1k', 'live_vqc', 'youtube_ugc_h264', 'lsvq_test', 'lsvq_test_1080p', 'finevd']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = process_logs(data_name, feats_name, model_name, log_path, select_criteria)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,cvd_2014,camp-vqa,0.9329 (±0.0202),0.7854 (±0.0329),0.9443 (±0.0310),7.0138 (±1.3861)
1,konvid_1k,camp-vqa,0.9267 (±0.0107),0.7657 (±0.0167),0.9361 (±0.0078),0.2283 (±0.0105)
2,live_vqc,camp-vqa,0.9221 (±0.0349),0.7566 (±0.0403),0.9404 (±0.0172),5.9295 (±0.7802)
3,youtube_ugc_h264,camp-vqa,0.9005 (±0.0138),0.7205 (±0.0209),0.9198 (±0.0116),0.2506 (±0.0132)
4,lsvq_test,camp-vqa,0.9193 (±0.0007),0.7604 (±0.0011),0.9328 (±0.0006),3.7891 (±0.0165)
5,lsvq_test_1080p,camp-vqa,0.9075 (±0.0016),0.7278 (±0.0022),0.9196 (±0.0017),4.0756 (±0.0408)
6,finevd,camp-vqa,0.9194 (±0.0305),0.7487 (±0.0402),0.9234 (±0.0347),4.9052 (±0.7558)


In [598]:
log_path = f'../log/best_model/'
feats_name = ['camp-vqa']
data_name = ['live_yt_gaming', 'kvq']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = process_logs(data_name, feats_name, model_name, log_path, select_criteria)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,live_yt_gaming,camp-vqa,0.9029 (±0.0215),0.7272 (±0.0331),0.9221 (±0.0270),6.5799 (±0.9131)
1,kvq,camp-vqa,0.9563 (±0.0041),0.8241 (±0.0071),0.9576 (±0.0035),0.1631 (±0.0054)


w/ finetune, train on LSVQ

In [599]:
is_finetune = True
select_criteria = 'byrmse'
log_path = f'../log/fine_tune/'
feats_name = ['camp-vqa']
data_name = ['cvd_2014', 'konvid_1k', 'live_vqc', 'youtube_ugc_h264', 'finevd', 'live_yt_gaming', 'kvq']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = read_csv_log(data_name, feats_name, log_path, select_criteria, is_finetune)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,cvd_2014,camp-vqa_finetune,0.966351,0.855689,0.963784,5.872598
1,konvid_1k,camp-vqa_finetune,0.930447,0.781007,0.943870,0.209175
2,live_vqc,camp-vqa_finetune,0.934294,0.777483,0.946193,5.042071
3,youtube_ugc_h264,camp-vqa_finetune,0.912128,0.745122,0.928009,0.229716
4,finevd,camp-vqa_finetune,0.924035,0.758330,0.933102,4.708112
5,live_yt_gaming,camp-vqa_finetune,0.905028,0.734174,0.941893,5.706617
6,kvq,camp-vqa_finetune,0.967406,0.849409,0.967187,0.142982


w/o finetune, train on LSVQ ----- cross-evaluation results of CAMP-VQA

In [600]:
is_finetune = False
select_criteria = 'byrmse'
log_path = f'../log/fine_tune/'
feats_name = ['camp-vqa']
data_name = ['cvd_2014', 'konvid_1k', 'live_vqc', 'youtube_ugc_h264', 'finevd', 'live_yt_gaming', 'kvq']
data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list = read_csv_log(data_name, feats_name, log_path, select_criteria, is_finetune)
df_results = create_results_dataframe(data_list, feats_list, srcc_list, krcc_list, plcc_list, rmse_list)
df_results

,DATASET,FEATS_NAME,SRCC (±STD),KRCC (±STD),PLCC (±STD),RMSE (±STD)
0,cvd_2014,camp-vqa_wo_finetune,0.906552,0.732342,0.933242,7.703348
1,konvid_1k,camp-vqa_wo_finetune,0.926325,0.764973,0.932354,0.231694
2,live_vqc,camp-vqa_wo_finetune,0.918964,0.751981,0.936905,5.963103
3,youtube_ugc_h264,camp-vqa_wo_finetune,0.879855,0.690704,0.897556,0.284665
4,finevd,camp-vqa_wo_finetune,0.864617,0.664544,0.889649,5.925250
5,live_yt_gaming,camp-vqa_wo_finetune,0.864246,0.663216,0.883604,7.921432
6,kvq,camp-vqa_wo_finetune,0.811067,0.615503,0.809681,0.331352


Predict Score (visualise Captions on FineVD)

In [601]:
# feature_name -> Dimension
def map_dimension(name):
    if not isinstance(name, str):
        return "Other"
    s = name.lower()
    if "color" in s:
        return "Color"
    if "noise" in s:
        return "Noise"
    if "temporal" in s:
        return "Temporal"
    if "blur" in s:
        return "Blur"
    if "artifact" in s:
        return "Artifact"
    if "camp-vqa.csv" in s and "mos_" not in s:
        return "Overall"
    return "Other"
def show_dimension(df, dim):
    subset = df[df["Dimension"] == dim].sort_values(by="vid").reset_index(drop=True)
    return subset
def show_video(df, vid):
    subset = df[df["vid"] == vid].sort_values(by="vid").reset_index(drop=True)
    order = ["Color", "Noise", "Artifact", "Blur", "Temporal", "Overall"]
    subset["Dimension"] = pd.Categorical(subset["Dimension"], categories=order, ordered=True)
    subset = subset.sort_values(by="Dimension").reset_index(drop=True)
    return subset
def load_predict_scores(csv_path):
    predict_df = pd.read_csv(csv_path)
    predict_df["Dimension"] = predict_df["feature_name"].map(map_dimension)
    out_df = predict_df[["vid", "Dimension", "MOS", "y_test_pred_logistic"]].rename(columns={"y_test_pred_logistic": "Predict_Score"})

    out_df["MOS"] = pd.to_numeric(out_df["MOS"], errors="coerce").round(2)
    out_df["Predict_Score"] = pd.to_numeric(out_df["Predict_Score"], errors="coerce").round(2)
    out = out_df.sort_values(by="vid").reset_index(drop=True)
    return out

csv_path = "../figs/captions_log/predict_score/TEST_camp-vqa_predictScore.csv"
out = load_predict_scores(csv_path)
# out.head()
dimension = show_dimension(out, "Overall")
dimension

,vid,Dimension,MOS,Predict_Score
0,0_02_06_1603525129-yase,Overall,64.69,62.06
1,0_03_00_1533467737,Overall,47.99,43.71
2,0_09_00_1529317714,Overall,68.65,61.91
3,0_16_07_500001604801190-yase,Overall,33.69,41.51


Predict Score (visualise FineVD)

In [602]:
csv_path = "../figs/captions_log/predict_score/FINEVD_TEST_camp-vqa_predictScore.csv"
out = load_predict_scores(csv_path)
# out dimension
# dimension = show_dimension(out, "Temporal")
# dimension
# out video
video = show_video(out, "0_15_07_1608740054-yase")
video

,vid,Dimension,MOS,Predict_Score
0,0_15_07_1608740054-yase,Color,68.73,68.01
1,0_15_07_1608740054-yase,Noise,65.46,62.89
2,0_15_07_1608740054-yase,Artifact,70.84,65.51
3,0_15_07_1608740054-yase,Blur,74.49,70.25
4,0_15_07_1608740054-yase,Temporal,70.30,64.01
5,0_15_07_1608740054-yase,Overall,75.10,71.43
